In [5]:
%pip install --upgrade --quiet langchain langchain-community langchain-text-splitters langchain-chroma>=0.1.2 langchain-openai pypdf chromadb==0.4.14 boto3

You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_community.document_loaders.pdf import PyPDFDirectoryLoader
from langchain.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import glob

DATA_PATH = '/Users/riju/Downloads/git_repo/rag_in_local/data/books/'

def load_documents(DATA_PATH):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1000,
        chunk_overlap = 500,
        length_function = len,
        add_start_index=True
    )
    loader = PyPDFDirectoryLoader(DATA_PATH, glob="*.pdf")
    chunks = loader.load_and_split(text_splitter=text_splitter)
    return chunks

chunks = load_documents(DATA_PATH)

In [3]:
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.embeddings.bedrock import BedrockEmbeddings
from langchain_openai import OpenAIEmbeddings

API_KEY = '<YOUR_API_KEY>'
def get_embedding_function(API_KEY):
    # embeddings = OllamaEmbeddings( base_url='http://192.168.0.200:11434',
    #     model="nomic-embed-text")
    embeddings = OpenAIEmbeddings(api_key= API_KEY)
    return embeddings

In [5]:
from langchain_chroma import Chroma
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
import os
import shutil

CHROMA_PATH = "chroma"
DATA_PATH = "data"

def add_to_chroma(chunks: list[Document], clear_data = False):
    if clear_data:
       clear_database() 

    # Load the existing database.
    db = Chroma(
        persist_directory=CHROMA_PATH, embedding_function=get_embedding_function(API_KEY)
    )

    # Calculate Page IDs.
    chunks_with_ids = calculate_chunk_ids(chunks)

    # Add or Update the documents.
    existing_items = db.get(include=[])  # IDs are always included by default
    existing_ids = set(existing_items["ids"])
    print(f"Number of existing documents in DB: {len(existing_ids)}")

    # Only add documents that don't exist in the DB.
    new_chunks = []
    for chunk in chunks_with_ids:
        if chunk.metadata["id"] not in existing_ids:
            new_chunks.append(chunk)

    if len(new_chunks):
        print(f"👉 Adding new documents: {len(new_chunks)}")
        new_chunk_ids = [chunk.metadata["id"] for chunk in new_chunks]
        db.add_documents(new_chunks, ids=new_chunk_ids)
        db.persist()
    else:
        print("✅ No new documents to add")


def calculate_chunk_ids(chunks):

    # This will create IDs like "data/monopoly.pdf:6:2"
    # Page Source : Page Number : Chunk Index

    last_page_id = None
    current_chunk_index = 0

    for chunk in chunks:
        source = chunk.metadata.get("source")
        page = chunk.metadata.get("page")
        current_page_id = f"{source}:{page}"

        # If the page ID is the same as the last one, increment the index.
        if current_page_id == last_page_id:
            current_chunk_index += 1
        else:
            current_chunk_index = 0

        # Calculate the chunk ID.
        chunk_id = f"{current_page_id}:{current_chunk_index}"
        last_page_id = current_page_id

        # Add it to the page meta-data.
        chunk.metadata["id"] = chunk_id

    return chunks


def clear_database():
    if os.path.exists(CHROMA_PATH):
        shutil.rmtree(CHROMA_PATH)


add_to_chroma(chunks)

/var/folders/2r/r83tm9k90b7518p7pwfx6sbr0000gn/T/ipykernel_56779/4054608653.py:15: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  db = Chroma(


Number of existing documents in DB: 0
👉 Adding new documents: 35


/var/folders/2r/r83tm9k90b7518p7pwfx6sbr0000gn/T/ipykernel_56779/4054608653.py:37: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


In [6]:
import argparse
# from dataclasses import dataclass
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain_community.llms.ollama import Ollama

PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Answer the question based on the above context: {question}
"""

def query_data(query_text):
    # Create CLI.
    # parser = argparse.ArgumentParser()
    # parser.add_argument("query_text", type=str, help="The query text.")
    # args = parser.parse_args()
    # query_text = args.query_text

    # Prepare the DB.
    embedding = get_embedding_function(API_KEY)
    db = Chroma(persist_directory=CHROMA_PATH, embedding_function=embedding)

    # Search the DB.
    results = db.similarity_search_with_relevance_scores(query_text, k=5)

    context_text = "\n\n---\n\n".join([doc.page_content for doc, _score in results])
    prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)
    prompt = prompt_template.format(context=context_text, question=query_text)
    # print(prompt)

    model = Ollama(model="mistral")
    response_text = model.invoke(prompt)

    sources = [doc.metadata.get("id", None) for doc, _score in results]
    formatted_response = f"Response: {response_text}\nSources: {sources}"
    print(formatted_response)
    return response_text

In [7]:
query_data("who is the writer of the book")

Response:  Derek Strange is the author of the book.
Sources: ['/Users/riju/Downloads/git_repo/rag_in_local/data/books/penguin_book.pdf:3:0', '/Users/riju/Downloads/git_repo/rag_in_local/data/books/penguin_book.pdf:4:0', '/Users/riju/Downloads/git_repo/rag_in_local/data/books/penguin_book.pdf:1:0', '/Users/riju/Downloads/git_repo/rag_in_local/data/books/penguin_book.pdf:33:0', '/Users/riju/Downloads/git_repo/rag_in_local/data/books/penguin_book.pdf:33:1']


' Derek Strange is the author of the book.'